In [43]:
import json
import os
import re
import string
import numpy as np
from gensim.models import Word2Vec

#**LOADING THE DATA**

In [44]:
from google.colab import files
import pandas as pd

In [45]:
uploaded = files.upload()
df = pd.read_csv('trimmed_data.csv', encoding = 'utf-8-sig', index_col=0)

Saving trimmed_data.csv to trimmed_data (1).csv


In [46]:
pos_comment = (df[df.tag == 'নিশ্চিত ইতিবাচক'])[:1000]
neg_comment = (df[df.tag == 'নিশ্চিত নেতিবাচক'])[:1000]

In [47]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d csoham/classification-bengali-news-articles-indicnlp
import os 
import zipfile 
local_dir='/content/classification-bengali-news-articles-indicnlp.zip'
zip_ref=zipfile.ZipFile(local_dir,'r')
zip_ref.extractall('/tmp')
zip_ref.close

Saving kaggle.json to kaggle (1).json
classification-bengali-news-articles-indicnlp.zip: Skipping, found more recently modified local copy (use --force to force download)


<bound method ZipFile.close of <zipfile.ZipFile filename='/content/classification-bengali-news-articles-indicnlp.zip' mode='r'>>

In [48]:
uploaded = files.upload()
df = pd.read_csv('SENTIMENT_DATASET_Modified.csv', encoding = 'utf-8-sig', index_col=0)

Saving SENTIMENT_DATASET_Modified.csv to SENTIMENT_DATASET_Modified (1).csv


#**PREPROCESSING**

In [56]:
#VOCAB
import json
import os
import re
import string
import numpy as np

from gensim.models import Word2Vec
def extract_text(filename, field):
    
    extracted_field=[]
    
    with open(os.path.join('data', filename), 'r') as f:
        articles=json.load(f)
    
    for article in articles['articles']:
        extracted_field.append(article[field].strip())
    
    return extracted_field

def replace_strings(texts, replace):
    new_texts=[]
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    
    for text in texts:
        for r in replace:
            text=text.replace(r[0], r[1])
        text=emoji_pattern.sub(r'', text)
        text=english_pattern.sub(r'', text)
        text=re.sub(r'\s+', ' ', text).strip()
        new_texts.append(text)

    return new_texts

def remove_punc(sentences):
    # import ipdb; ipdb.set_trace()
    new_sentences=[]
    exclude = list(set(string.punctuation))
    exclude.extend(["’", "‘", "—"])
    for sentence in sentences:
        s = ''.join(ch for ch in sentence if ch not in exclude)
        new_sentences.append(s)
    
    return new_sentences

replace=[('\u200c', ' '),
         ('\u200d', ' '),
        ('\xa0', ' '),
        ('\n', ' '),
        ('\r', ' ')]

In [57]:
abz_body=extract_text('/tmp/News Articles/anandabazar_articles.txt', 'body')
abz_body=remove_punc(abz_body)
abz_body=replace_strings(abz_body, replace)

zee_body=extract_text('/tmp/News Articles/zeenews_articles.txt', 'body')
zee_body=remove_punc(zee_body)
zee_body=replace_strings(zee_body, replace)

ebala_body=extract_text('/tmp/News Articles/ebala_articles.txt', 'body')
ebala_body=remove_punc(zee_body)
ebala_body=replace_strings(zee_body, replace)

kaggle_gold=remove_punc(df['data'])
ebala_body=replace_strings(kaggle_gold, replace)

body=[]
body.extend(zee_body)
body.extend(abz_body)
body.extend(ebala_body)
body.extend(kaggle_gold)

In [59]:
print(f"Total Number of training data: {len(body)}")

Total Number of training data: 35015


In [60]:
body=[article.split('।') for article in body]
body=[item for sublist in body for item in sublist]
body=[item.strip() for item in body if len(item.split())>1]
body=[item.split() for item in body]

**GENERATING THE MODEL & VOCABULARY**

In [61]:
model = Word2Vec(body, size=200, window=5, min_count=1)

**TRAIN TEST SPLIT**

In [122]:
from sklearn.model_selection import train_test_split

y = np.concatenate((np.ones(len(pos_comment)), np.zeros(len(neg_comment))))
x_train, x_test, y_train, y_test = train_test_split(np.concatenate((pos_comment, pos_comment)), y, test_size=0.2)

#Do some very minor text preprocessing
n_dim = 200
#TRAIN model and build vocab
model = Word2Vec(body, size=200, window=5, min_count=1)

**TRAINING & VALIDATION**

In [123]:
def buildWordVector(text, size):
  embedding_vector = np.zeros(size).reshape((1, size))
  count = 0
  for word in text:
    if word in model.wv.vocab:
      count += 1
      embedding_vector += model.wv[word]
  if count != 0:
    embedding_vector = embedding_vector/count
  #return embedding_vector.astype(np.float32)
  return embedding_vector

In [124]:
from sklearn.preprocessing import scale
train_vecs = np.concatenate([buildWordVector(z, n_dim) for z in x_train])
train_vecs = scale(train_vecs)

In [125]:
#Build test tweet vectors then scale
test_vecs = np.concatenate([buildWordVector(z, n_dim) for z in x_test])
test_vecs = scale(test_vecs)

In [126]:
train_dataset = list(zip(train_vecs,y_train))
test_dataset = list(zip(test_vecs,y_test))

In [169]:
print(len(y_train))

1600


In [129]:
#Use classification algorithm (i.e. Stochastic Logistic Regression) on training set, then assess model performance on test set
from sklearn.linear_model import SGDClassifier
lr = SGDClassifier(loss='log', penalty='l1')
lr.fit(train_vecs, y_train)
print ('Test Accuracy: %.2f'%lr.score(test_vecs, y_test))

Test Accuracy: 0.52


In [133]:
#Use classification algorithm (i.e. Stochastic Logistic Regression) on training set, then assess model performance on test set
from sklearn.linear_model import SGDClassifier
lr = SGDClassifier(loss='log', penalty='l2')
lr.fit(train_vecs, y_train)
print ('Test Accuracy: %.2f'%lr.score(test_vecs, y_test))

Test Accuracy: 0.48


In [137]:
#Use classification algorithm (i.e. Stochastic Logistic Regression) on training set, then assess model performance on test set
from sklearn.linear_model import SGDClassifier
lr = SGDClassifier(loss='log', penalty='elasticnet')
lr.fit(train_vecs, y_train)
print ('Test Accuracy: %.2f'%lr.score(test_vecs, y_test))

Test Accuracy: 0.52


#**Saving the Model**

In [ ]:
#model.wv.save_word2vec_format('final_project_model_text.txt', binary=False)
model.wv.save_word2vec_format('final_project_model_binary.txt', binary=True)

#**Validation**

In [150]:
#Validation
uploaded = files.upload()
df2 = pd.read_csv('validation.csv', encoding = 'utf-8-sig', index_col=0)

Saving validation.csv to validation.csv


In [180]:
validation = df['data']
validation_score = np.concatenate((np.ones(200), np.zeros(200)))
validation_vecs = np.concatenate([buildWordVector(z, n_dim) for z in validation])
validation_vecs = scale(test_vecs)

In [181]:
from sklearn.linear_model import SGDClassifier
lr = SGDClassifier(loss='log', penalty='elasticnet')
lr.fit(train_vecs, y_train)
print ('Test Accuracy: %.2f'%lr.score(validation_vecs, validation_score))

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='elasticnet',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
model.wv.most_similar('ছেলে', topn=5)

What are the words most similar to chele


[('মেয়ে', 0.9196559190750122),
 ('বাবা', 0.9086806774139404),
 ('স্ত্রী', 0.8819312453269958),
 ('স্বামী', 0.8757562637329102),
 ('সন্তান', 0.8697322010993958)]

**NAME GENERATOR**

In [ ]:
model.wv.most_similar('মাহফুজ', topn=5)

[('খ্যাত', 0.9400482177734375),
 ('ঝিল্লির', 0.9354316592216492),
 ('অনুপস্থিতিতে', 0.932100772857666),
 ('জায়া', 0.9279686212539673),
 ('শেখাও', 0.9239286184310913)]

**NN**

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# Hyperparameters
batch_size = 100
num_iters = 3000
input_dim = 200
num_hidden = 1000
output_dim = 2

learning_rate = 0.1

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


num_epochs = num_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)  

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False) 

class DeepNeuralNetworkModel(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden):
        super().__init__()
        ### 1st hidden layer: 784 --> 100
        self.linear_1 = nn.Linear(input_size, num_hidden)
        ### Non-linearity in 1st hidden layer
        self.relu_1 = nn.ReLU()

        ### 2nd hidden layer: 100 --> 100
        self.linear_2 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 2nd hidden layer
        self.relu_2 = nn.ReLU()

        ### 3rd hidden layer: 100 --> 100
        self.linear_3 = nn.Linear(num_hidden, num_hidden)
        ### Non-linearity in 3rd hidden layer
        self.relu_3 = nn.ReLU()

        ### Output layer: 100 --> 10
        self.linear_out = nn.Linear(num_hidden, num_classes)

    def forward(self, x):
        ### 1st hidden layer
        out  = self.linear_1(x.float())
        ### Non-linearity in 1st hidden layer
        out = self.relu_1(out)
        
        ### 2nd hidden layer
        out  = self.linear_2(out)
        ### Non-linearity in 2nd hidden layer
        out = self.relu_2(out)

        ### 3rd hidden layer
        out  = self.linear_3(out)
        ### Non-linearity in 3rd hidden layer
        out = self.relu_3(out)
        
        # Linear layer (output)
        probas  = self.linear_out(out)
        return probas

# INSTANTIATE MODEL CLASS

model = DeepNeuralNetworkModel(input_size = input_dim,
                               num_classes = output_dim,
                               num_hidden = num_hidden)
# To enable GPU
#model = model.float()
model.to(device)

# INSTANTIATE LOSS & OPTIMIZER CLASS
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):

        images = images.view(-1, 200).to(device)
        labels = labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images.float()) 

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        iter += 1

        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
               
                images = images.to(device)

                # Forward pass only to get logits/output
                outputs = model(images.float())

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs, 1)

                # Total number of labels
                total += labels.size(0)


                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() 
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct.item() / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))